In [2]:
!ls /kaggle/input/new-plant-diseases-dataset/New\ Plant\ Diseases\ Dataset\(Augmented\)/'New Plant Diseases Dataset(Augmented)'

ls: /kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented): No such file or directory


In [6]:
import os
import pickle
import numpy as np
import cv2
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy'

In [1]:
img_size=(256,256)
n_img=100
root_dir="/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir=os.path.join(root_dir,"train")
val_dir=os.path.join(root_dir,"valid")

NameError: name 'os' is not defined

In [4]:
def convert(img_dir):
    try:
        img=cv2.imread(img_dir)
        if img is not None:
            img=cv2.resize(img,img_size)
            return img_to_array(img)
        else:
            return np.array([])
    except:
        print("Error!")
        return None

In [ ]:
img_list=[]
label_list=[]
try:
    print("Loading images...")
    diseases_folders_list=os.listdir(train_dir)
    for diseases_folder in diseases_folders_list:
        print(f"Processing {diseases_folder}...")
        diseases_images_list=os.listdir(f"{train_dir}/{diseases_folder}/")
        for image in diseases_images_list[:n_img]:
            image_directory=f"{train_dir}/{diseases_folder}/{image}"
            if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True:
                img_list.append(convert(image_directory))
                label_list.append(diseases_folder)
    print("Images processed!")
except Exception as e:
    print(f"Error:{e}")
np_img_list=np.array(img_list,dtype=np.float16)/225.0
print("Number of images processed:",len(img_list))

In [ ]:
lb=LabelBinarizer()
img_labels=lb.fit_transform(label_list)
pickle.dump(lb,open("plant_disease_label_transform.pkl","wb"))
print("Number of classes:",len(lb.classes_))

In [7]:
x_train,x_test,y_train,y_test=train_test_split(np_img_list,img_labels,test_size=0.2,random_state=42)

In [8]:
augment=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [9]:
EPOCHS=25
steps=100
LR=1e-3
bs=32
height=256
width=256
depth=3

In [ ]:
model=Sequential()
inputShape=(height,width,depth)
chanDim=-1
if K.image_data_format()=="channels_first":
    inputShape=(depth,height,width)
    chanDim=1
model.add(Conv2D(32,(3,3),padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(len(lb.classes_)))
model.add(Activation('softmax'))
model.summary()

In [ ]:
o=Adam(lr=LR,decay=LR/EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=o,metrics=["accuracy"])
print("Training model...")
history=model.fit_generator(augment.flow(x_train,y_train,batch_size=bs),validation_data=(x_test,y_test),steps_per_epoch=len(x_train)//bs,epochs=EPOCHS,verbose=1)

In [ ]:
accuracy=history.history["accuracy"]
validation_accuracy=history.history["val_accuracy"]
loss=history.history["loss"]
validation_loss=history.history["val_loss"]
epochs=range(1,len(accuracy)+1)
plt.plot(epochs,accuracy,"b",label="Training Accuracy")
plt.plot(epochs,validation_accuracy,"r",label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs,loss,"b",label="Training Loss")
plt.plot(epochs,validation_loss,"r",label="Validation Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
print("Calculating model accuracy...")
score=model.evaluate(x_test,y_test)
print(f"Test_Accuracy:{score[1]*100}")

In [ ]:
!wget https://ohioline.osu.edu/sites/ohioline/files/imce/Plant_Pathology/PLNTPTH-CER-02_Figure_1.png

In [ ]:
!ls

In [17]:
def predict(img_path):
    img_arr=convert(img_path)
    np_img=np.array(img_arr,dtype=np.float16)/255.0
    np_img=np.expand_dims(np_img,0)
    fig,ax=plt.subplots(1,1)
    result=model.predict_classes(np_img)
    plt.imshow(plt.imread(img_path))
    print("Predicted:",(lb.classes_[result][0]))
    plt.show()

In [ ]:
import glob
images=glob.glob("*.png")
for img in images:
    predict(img)